# Machine Learning with School Budgets

This notebook is inspired by the DataCamp course "Machine Learning with the Experts: School Budgets", which applies machine learning concepts to a real Data Driven competition on school budgets.

Competition: https://www.drivendata.org/competitions/46/box-plots-for-education-reboot/
        
Course: https://www.datacamp.com/courses/machine-learning-with-the-experts-school-budgets

 ## Goal: 
To correctly label budget line items by training a supervised model to predict the probability of each possible label, taking most probable label as the correct label.

### Approach:

Classification Algorithm


Specifically, we have ourselves a multi-class-multi-label classification problem (quite a mouthful!), because there are 9 broad categories that each take on many possible sub-label instances.

In [1]:
## Importing necessary modules
import pandas as pd

import numpy as np

import multilabel_sample as mls                         # ***See footnote below

# Import random forest classifer
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.multiclass import OneVsRestClassifier      # treats each column of y independently
                                                        # fits a seperate classifier for each of the columns
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import FunctionTransformer

from sklearn.pipeline import FeatureUnion

# creates bag of word representation
from sklearn.feature_extraction.text import CountVectorizer

# very useful for memory, large amount of text data
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.preprocessing import Imputer

from sklearn.feature_selection import chi2, SelectKBest

from sklearn.preprocessing import MaxAbsScaler

# visualizations
import matplotlib.pyplot as plt
%matplotlib inline

# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

\* The first step in modeling is to split the data into a training set and a test set. Some labels don't occur very often, but we want to make sure that they appear in both the training and the test sets. We provide a function that will make sure at least min_count examples of each label appear in each split: multilabel_train_test_split

### Loading and Inspecting Data

In [4]:
# loading data into dataframe
df = pd.read_csv("Data/TrainingData.csv", index_col=0)

In [5]:
# inspecting first five rows
df.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


### Filtering Data

In [6]:
# Text pattern for tokenizing later on
alphanumeric_tokens = '[A-Za-z0-9]+(?=\\s+)'

In [8]:
# Selecting numeric columns
numeric_columns = ['FTE', 'Total']

# Creating a new DataFrame
#numbers = df[numbers].fillna(-1000)

# For categorizing data
labels = ['Function','Use', 'Sharing','Reporting',
          'Student_Type', 'Position_Type','Object_Type',
          'Pre_K', 'Operating_Status']

# For all other columns
non_labels = [c for c in df.columns if c not in labels]

In [11]:
# defining sample size
sample_size = 40000

# taking sample from data
sampling = mls.multilabel_sample_dataframe(df,
                                       pd.get_dummies(df[labels]),
                                       size=sample_size,
                                       min_count=25,
                                       seed=43)

dummy_labels = pd.get_dummies(sampling[labels])

### Preprocessing Function

In [12]:
def combine_text_columns(data_frame, to_drop=numeric_columns + labels):
    """ Takes the dataset as read in, drops the non-feature, non-text columns and
        then combines all of the text columns into a single vector that has all of
        the text for a row.
        
        :param data_frame: The data as read in with read_csv (no preprocessing necessary)
        :param to_drop (optional): Removes the numeric and label columns by default.
    """
    # drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    
    # replace nans with blanks
    text_data.fillna("", inplace=True)
    
    # joins all of the text items in a row (axis=1)
    # with a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

## ML Pipeline

In [13]:
## Splitting data up into training and testing

X_train, X_test, y_train, y_test = mls.multilabel_train_test_split(sampling[non_labels],
                                                               dummy_labels,
                                                               0.2,
                                                               min_count=3,
                                                               seed=43)

In [14]:
# numeric data that needs imputation, 
# and text data that needs to be converted into a bag-of-words

# takes df returns text column(s)
get_text_data = FunctionTransformer(combine_text_columns, validate=False)
# doesnt need to check for NaN's or verify dtypes (validate=False)

# takes df returns numeric column(s)
get_numeric_data = FunctionTransformer(lambda x: x[numeric_columns], validate=False)

In [21]:
## Creating pipeline

# set a reasonable number of features before adding interactions
chi_k = 300
# create the pipeline object
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', HashingVectorizer(token_pattern=alphanumeric_tokens,
                                                     non_negative=True, norm=None, binary=False,
                                                     ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
 
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(RandomForestClassifier()))
    ])

Now that a pipeline is set up, I can choose whatever classification model I want and fine tune the parameters to improve the performace of the model!

### Fitting Model

In [22]:
# Fit to the training data
pl.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False,
          func=<function <lambda> at 0x7f2a373e0730>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y='dep... oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          n_jobs=1))])

In [23]:
# Compute and print accuracy
accuracy = pl.score(X_test, y_test)

print("\nAccuracy on budget dataset: ", accuracy)


Accuracy on budget dataset:  0.729125


### Predicting

In [27]:
# Load holdout data
holdout = pd.read_csv("Data/TestData.csv", index_col=0)

# Make predictions
predictions = pl.predict_proba(holdout)

# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=pd.get_dummies(df[labels]).columns,
                             index=holdout.index,
                             data=predictions)


In [29]:
prediction_df.head(10)

,Function_Aides Compensation,Function_Career & Academic Counseling,Function_Communications,Function_Curriculum Development,Function_Data Processing & Information Services,Function_Development & Fundraising,Function_Enrichment,Function_Extended Time & Tutoring,Function_Facilities & Maintenance,Function_Facilities Planning,...,Object_Type_Rent/Utilities,Object_Type_Substitute Compensation,Object_Type_Supplies/Materials,Object_Type_Travel & Conferences,Pre_K_NO_LABEL,Pre_K_Non PreK,Pre_K_PreK,Operating_Status_Non-Operating,"Operating_Status_Operating, Not PreK-12",Operating_Status_PreK-12 Operating
180042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.3,0.0,0.9,0.4,0.0,0.0,0.0,1.0
28872,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.1,0.0,0.0,...,0.0,0.0,0.3,0.0,0.8,0.5,0.0,0.0,0.0,0.9
186915,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.9,0.2,0.0,0.0,0.1,0.7
412396,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.9,0.2,0.0,0.0,0.1,0.7
427740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1,0.0,0.7,0.2,0.0,0.1,0.0,1.0
69847,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.8,0.0,0.9,0.0,0.1,0.0,0.0,1.0
358824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.0,0.1,0.0,1.0,0.0,0.0,0.0,0.0,0.9
254148,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.4,0.3,0.0,0.0,0.0,0.9
296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9,0.0,0.8,0.4,0.0,0.0,0.1,1.0
416755,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1,0.4,0.6,0.0,0.1,0.0,0.8


### Writing to File

In [30]:
# Save prediction_df to csv called "predictions.csv"
prediction_df.to_csv("predictions.csv")